In [29]:
import matplotlib, cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import math
import os
import itertools
import random
%matplotlib inline

In [3]:
task_2_train = "Task2Dataset/Training/png/"
task_2_test = "Task2Dataset/TestWithoutRotations/images/"
task_2_test_annotations = "Task2Dataset/TestWithoutRotations/annotations/"

task_3_test_annotations = "Task3AdditionalTestDataset/annotations/"
task_3_test = "Task3AdditionalTestDataset/images/"

In [4]:
def get_train_features(train_folder):
    
    train_points = {}
    train_descriptors = {}
    
    sift = cv2.SIFT_create(nOctaveLayers=5, nfeatures=1000, contrastThreshold=0.1)

    for train_image in os.listdir(train_folder):
        image = cv2.imread(os.path.join(train_folder, train_image), 0)
        
        pt, desc = sift.detectAndCompute(image, None)
        
        train_points[train_image] = pt
        train_descriptors[train_image] = desc

    return train_descriptors, train_points


In [24]:
def get_annotations(annotation_folder):
    annotations = {}

    for annotation_file in os.listdir(annotation_folder):
        image_name = os.path.basename(annotation_file)
        annotation_file = open(os.path.join(annotation_folder, annotation_file), "r")
        annotation_lines = annotation_file.readlines()
        image_annot = []
        for line in annotation_lines:
            image_annot.append(line.split(",")[0])
        annotations[image_name] = image_annot
    return annotations



In [5]:
def sift_matches(train_pt, train_desc, test_image_path):

    test_image = cv2.imread(test_image_path, 0)

    sift = cv2.SIFT_create()

    test_points, test_descriptor = sift.detectAndCompute(test_image, None)

    matcher = cv2.BFMatcher_create()

    matches = matcher.knnMatch(train_desc, test_descriptor, 2)
    retained_matches = []
    for one, two in matches:
        #change threshold; lower to reduce matches retained
        if one.distance < 0.3*two.distance:
            retained_matches.append([one])

    return retained_matches
 

In [49]:
def evaluate(train_folder, test_image_folder, test_annotation_folder):
    total_total = 0
    total_correct = 0
    for test_image in os.listdir(test_image_folder):
        
        train_pts, train_desc = get_train_features(train_folder)
        
        scores = {}
        
        for image, desc in train_desc.items():

            matches = sift_matches(train_desc[image], train_pts[image], os.path.join(test_image_folder, test_image))
            scores[image.split("-")[1].split(".")[0]] = len(matches)

        scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))
        annotations = get_annotations(test_annotation_folder)

        scores = dict(itertools.islice(scores.items(), len(annotations)))

        # note - need to refactor task 3 test annotations from .csv to .txt before use
        test_annotations = annotations[test_image.split(".")[0]+".txt"]
        true_positives, false_positives, true_negatives, false_negatives = 0,0,0,0 
        for annotation in test_annotations:
            if annotation in scores:
                true_positives += 1
            else:
                false_positives += 1
                false_negatives += 1
        total = false_positives + true_positives
        total_total+=total
        total_correct+=true_positives
        print(test_image, ":", true_positives, "/", total)
    print("Total performance:", total_correct, "/", total_total)
                


In [50]:
evaluate(task_2_train, task_2_test, task_2_test_annotations)

test_image_1.png : 3 / 4
test_image_10.png : 4 / 5
test_image_11.png : 3 / 4
test_image_12.png : 3 / 5
test_image_13.png : 4 / 5
test_image_14.png : 2 / 4
test_image_15.png : 2 / 4
test_image_16.png : 3 / 5
test_image_17.png : 5 / 5
test_image_18.png : 4 / 5
test_image_19.png : 4 / 4
test_image_2.png : 4 / 5
test_image_20.png : 4 / 4
test_image_3.png : 2 / 4
test_image_4.png : 4 / 5
test_image_5.png : 1 / 5
test_image_6.png : 2 / 4
test_image_7.png : 4 / 5
test_image_8.png : 3 / 5
test_image_9.png : 4 / 4
Total performance: 65 / 91


In [52]:
evaluate(task_2_train, task_3_test, task_3_test_annotations)

test_image_1.png : 3 / 4
test_image_10.png : 2 / 4
test_image_11.png : 3 / 4
test_image_12.png : 2 / 4
test_image_13.png : 2 / 4
test_image_14.png : 2 / 5
test_image_15.png : 3 / 4
test_image_16.png : 3 / 4
test_image_17.png : 2 / 4
test_image_18.png : 2 / 5
test_image_19.png : 2 / 4
test_image_2.png : 4 / 4
test_image_20.png : 3 / 4
test_image_3.png : 2 / 4
test_image_4.png : 3 / 5
test_image_5.png : 3 / 5
test_image_6.png : 5 / 5
test_image_7.png : 4 / 5
test_image_8.png : 3 / 4
test_image_9.png : 2 / 4
Total performance: 55 / 86
